Plastic contamination is a serious problem world wide and this study will create a model to forecast the amount of plastic in the near future. We will use Deep Learning to try and forecast plastic production. We  will use a simple recurrent neural network and a lstm model with 2 different hyperparameters and architectures.

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore')
import seaborn as sns
plt.style.use('fivethirtyeight')
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Activation, Dropout
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('../input/plastic-datasets/global-plastics-production.csv')
print(data.shape)
print(data.head())

In [ ]:
data.isnull().sum()

In [ ]:
col= ['Year', 'Global plastics production (million tonnes)']
ndata= data[col]
ndata.set_index(ndata.Year,inplace=True)
ndata.drop(columns=['Year'], inplace=True)
ndata.head()

In [ ]:
ndata.plot()

In [ ]:
def get_keras_format_series(series):
    """
    Convert a series to a numpy array of shape 
    [n_samples, time_steps, features]
    """
    
    series = np.array(series)
    return series.reshape(series.shape[0], series.shape[1], 1)


In [ ]:
mm=MinMaxScaler()
ndata= mm.fit_transform(ndata)

In [ ]:
test_years= 18
train = ndata[:-test_years] 
test = ndata[-test_years:] 
print(train.shape)
train_X, train_y = [], []
for i in range(0, train.shape[0]-6): 
    train_X.append(train[i:i+6]) 
    train_y.append(train[i+6:i+7])
    
train_X = get_keras_format_series(train_X) 
train_y = np.array(train_y)

input_years=9
test_X_init = test[:input_years] 
test_y = test[input_years:]
print(test_X_init.shape)

In [ ]:
cell_units = 50
epochs= 3000
    # initialize model
model = Sequential() 

model.add(SimpleRNN(cell_units, input_shape=(train_X.shape[1],1)))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_X, train_y, epochs=epochs, batch_size=64, verbose=0)

In [ ]:
def predict(X_init, n_steps, model):
    """
    Given an input series matching the model's expected format,
    generates model's predictions for next n_steps in the series      
    """
    
    X_init = X_init.copy().reshape(1,-1,1)
    preds = []
    
    # iteratively take current input sequence, generate next step pred,
    # and shift input sequence forward by a step (to end with latest pred).
    # collect preds as we go.
    for _ in range(n_steps):
        pred = model.predict(X_init)
        preds.append(pred)
        X_init[:,:-1,:] = X_init[:,1:,:] # replace first 11 values with 2nd through 12th
        X_init[:,-1,:] = pred # replace 12th value with prediction
    
    preds = np.array(preds).reshape(-1,1)
    
    return preds

def predict_and_plot(X_init, y, model, title):
    """
    Given an input series matching the model's expected format,
    generates model's predictions for next n_steps in the series,
    and plots these predictions against the ground truth for those steps 
    
    arguments
    ---------
    X_init (array): initial sequence, must match model's input shape
    y (array): true sequence values to predict, follow X_init
    model (keras.models.Sequential): trained neural network
    title (string): plot title   
    """
    
    y_preds = predict(test_X_init, n_steps=len(y), model=model) # predict through length of y
    
    
    # Below ranges are to set x-axes
    start_range = range(1, test_X_init.shape[0]+1) #starting at one through to length of test_X_init to plot X_init
    predict_range = range(test_X_init.shape[0], test_years)  #predict range is going to be from end of X_init to length of test_hours
    
    #using our ranges we plot X_init
    plt.plot(start_range, test_X_init)
    #and test and actual preds
    plt.plot(predict_range, test_y, color='orange')
    plt.plot(predict_range, y_preds, color='teal', linestyle='--')
    
    plt.title(title)
    plt.legend(['Initial Series','Target Series','Predictions'])

In [ ]:
predict_and_plot(test_X_init, test_y, model,
                 'Global Plastic: Test and predictions')
plt.savefig('model1.png')

In [ ]:
model.summary()

In [ ]:
cell_units= 70

model2 = Sequential() 
    
model2.add(LSTM(cell_units, input_shape=(train_X.shape[1],1))) 
   
model2.add(Dense(25))
model2.add(Dense(1))
    
model2.compile(loss='mean_squared_error', optimizer='adam')
model2.fit(train_X, train_y, epochs=epochs, batch_size=64, verbose=0)
    

In [ ]:
predict_and_plot(test_X_init, test_y, model2, 
                 'Global Plastic: Test and predictions')

In [ ]:
model2.summary()

In [ ]:
cell_units= 100

model3 = Sequential() 
    
model3.add(LSTM(cell_units, input_shape=(train_X.shape[1],1))) 
model3.add(Dropout(0.2))
model3.add(Dense(25))
model3.add(Dense(1))
    
model3.compile(loss='mean_squared_error', optimizer='adam')
model3.fit(train_X, train_y, epochs=epochs, batch_size=64, verbose=0)
    

In [ ]:

predict_and_plot(test_X_init, test_y, model3, 
                 'Global Plastic: Test and predictions')

In [ ]:
model3.summary()